# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.106792e+02     1.719980e+00
 * time: 0.1186380386352539
     1     1.009146e+01     9.315795e-01
 * time: 0.9719219207763672
     2    -1.170573e+01     1.113803e+00
 * time: 1.1092529296875
     3    -3.403988e+01     8.614626e-01
 * time: 1.2720279693603516
     4    -4.748347e+01     6.091410e-01
 * time: 1.4220459461212158
     5    -5.688204e+01     2.025914e-01
 * time: 1.5667710304260254
     6    -5.976214e+01     1.344801e-01
 * time: 1.680103063583374
     7    -6.088585e+01     5.378643e-02
 * time: 1.7917490005493164
     8    -6.133260e+01     4.756911e-02
 * time: 1.9084501266479492
     9    -6.159424e+01     4.625756e-02
 * time: 2.0237491130828857
    10    -6.181963e+01     2.822852e-02
 * time: 2.1334049701690674
    11    -6.198917e+01     2.131999e-02
 * time: 2.248522996902466
    12    -6.206661e+01     1.663200e-02
 * time: 2.3626670837402344
    13    -6.212623e+01     1.278014e-02
 * time: 2.469767093658447

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557686
    AtomicNonlocal      14.8522658
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666455616
